In [1]:
import findspark

In [2]:
findspark.init('/Users/kiranrudresha/Documents/spark/spark-2.2.0-bin-hadoop2.7')

In [6]:
from pyspark.sql import SparkSession

In [8]:
from pyspark.ml.regression import LinearRegression

In [9]:
spark = SparkSession.builder.appName('ship_build').getOrCreate()

In [10]:
ship = spark.read.csv('/Users/kiranrudresha/Documents/spark/pyspark/Python-and-Spark-for-Big-Data-master/'
                 'Spark_for_Machine_Learning/Linear_Regression/cruise_ship_info.csv',
                  inferSchema=True,header=True)

In [11]:
ship.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [13]:
for i in ship.head(5):
    print(i)
    print('/n')

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)
/n
Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)
/n
Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7)
/n
Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1)
/n
Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0)
/n


In [15]:
ship.groupBy('cruise_line').count().show()

+-----------------+-----+
|      cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [16]:
ship.count()

158

In [17]:
from pyspark.ml.feature import StringIndexer

In [23]:
Indexer = StringIndexer(inputCol='Cruise_line', outputCol='cruise_cat')

In [25]:
type(Indexer)

pyspark.ml.feature.StringIndexer

In [32]:
indexed = Indexer.fit(ship).transform(ship)

In [33]:
type(indexed)

pyspark.sql.dataframe.DataFrame

In [34]:
indexed.head(5)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0),
 Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7, cruise_cat=1.0),
 Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1, cruise_cat=1.0),
 Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0, cruise_cat=1.0)]

In [39]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [40]:
indexed.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'cruise_cat']

In [41]:
assembler = VectorAssembler(inputCols= ['Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'cruise_cat'], outputCol='features'
)

In [43]:
df1= assembler.transform(indexed)

In [44]:
df1.head(5)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0, features=DenseVector([6.0, 30.277, 6.94, 5.94, 3.55, 42.64, 16.0])),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0, features=DenseVector([6.0, 30.277, 6.94, 5.94, 3.55, 42.64, 16.0])),
 Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7, cruise_cat=1.0, features=DenseVector([26.0, 47.262, 14.86, 7.22, 7.43, 31.8, 1.0])),
 Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1, cruise_cat=1.0, features=DenseVector([11.0, 110.0, 29.74, 9.53, 14.88, 36.99, 1.0])),
 Row(Ship_name='Destin

In [46]:
output = df1.select('features','crew')

In [47]:
train_data,test_data=output.randomSplit([0.7,0.3])

In [49]:
train_data.count()

104

In [50]:
test_data.count()

54

In [51]:
lr = LinearRegression(labelCol='crew')

In [52]:
type(lr)

pyspark.ml.regression.LinearRegression

In [53]:
mod1= lr.fit(train_data)

In [54]:
type(mod1)

pyspark.ml.regression.LinearRegressionModel

In [55]:
fin_res = mod1.evaluate(test_data)

In [56]:
type(fin_res)

pyspark.ml.regression.LinearRegressionSummary

In [57]:
fin_res.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|0.047703429374529094|
|   -1.42203777944888|
| -0.9648237047218107|
| -1.4158461658146528|
|  0.3413433057278574|
|-0.44679950474637664|
| -0.7762542889450685|
|-0.24027181411136844|
| 0.26876194932143704|
|  0.7483569582630007|
| -0.6441380056365311|
| -0.8452092982856403|
| -0.6313772334211407|
|-0.46183852986246343|
|0.020226233195010934|
| -0.5074509138299224|
| -0.3854655447908861|
|-0.31460217960384895|
| -0.8894785447572389|
| -0.3511759624964963|
+--------------------+
only showing top 20 rows



In [58]:
fin_res.r2

0.9301993266571963

In [59]:
fin_res.rootMeanSquaredError

0.8848081679177828

In [60]:
df1.describe().show()

+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+
|summary|Ship_name|Cruise_line|               Age|           Tonnage|       passengers|           length|            cabins|passenger_density|             crew|       cruise_cat|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+
|  count|      158|        158|               158|               158|              158|              158|               158|              158|              158|              158|
|   mean| Infinity|       null|15.689873417721518| 71.28467088607599|18.45740506329114|8.130632911392404| 8.830000000000005|39.90094936708861|7.794177215189873|5.063291139240507|
| stddev|      NaN|       null| 7.615691058751413|37.229540025907866|9.677094775143416|1.793473548054825|

In [64]:
from pyspark.sql.functions import corr

In [66]:
df1.select(corr('crew','cabins')).show()

+------------------+
|corr(crew, cabins)|
+------------------+
|0.9508226063578497|
+------------------+



In [67]:
unlabel = test_data.select('features')

In [68]:
unlabel.show()

+--------------------+
|            features|
+--------------------+
|[4.0,220.0,54.0,1...|
|[5.0,86.0,21.04,9...|
|[6.0,30.276999999...|
|[6.0,90.0,20.0,9....|
|[6.0,110.23899999...|
|[6.0,112.0,38.0,9...|
|[7.0,116.0,31.0,9...|
|[7.0,158.0,43.7,1...|
|[8.0,77.499,19.5,...|
|[8.0,91.0,22.44,9...|
|[8.0,110.0,29.74,...|
|[9.0,85.0,19.68,9...|
|[9.0,110.0,29.74,...|
|[10.0,58.825,15.6...|
|[10.0,77.0,20.16,...|
|[10.0,81.76899999...|
|[10.0,90.09,25.01...|
|[10.0,91.62700000...|
|[11.0,85.0,18.48,...|
|[11.0,86.0,21.24,...|
+--------------------+
only showing top 20 rows



In [69]:
fin1 = mod1.transform(unlabel)

In [70]:
fin1.head(5)

[Row(features=DenseVector([4.0, 220.0, 54.0, 11.82, 27.0, 40.74, 0.0]), prediction=20.95229657062547),
 Row(features=DenseVector([5.0, 86.0, 21.04, 9.36, 10.22, 40.87, 3.0]), prediction=9.42203777944888),
 Row(features=DenseVector([6.0, 30.277, 6.94, 5.94, 3.55, 42.64, 16.0]), prediction=4.5148237047218105),
 Row(features=DenseVector([6.0, 90.0, 20.0, 9.64, 10.29, 45.0, 15.0]), prediction=10.415846165814653),
 Row(features=DenseVector([6.0, 110.239, 37.0, 9.51, 14.87, 29.79, 1.0]), prediction=11.158656694272143)]

In [71]:
test_data.head(5)

[Row(features=DenseVector([4.0, 220.0, 54.0, 11.82, 27.0, 40.74, 0.0]), crew=21.0),
 Row(features=DenseVector([5.0, 86.0, 21.04, 9.36, 10.22, 40.87, 3.0]), crew=8.0),
 Row(features=DenseVector([6.0, 30.277, 6.94, 5.94, 3.55, 42.64, 16.0]), crew=3.55),
 Row(features=DenseVector([6.0, 90.0, 20.0, 9.64, 10.29, 45.0, 15.0]), crew=9.0),
 Row(features=DenseVector([6.0, 110.239, 37.0, 9.51, 14.87, 29.79, 1.0]), crew=11.5)]